

```
 There are 2 basic uses for the pre-trained models!
```


1st : use a trained model directly ro image recognition

2nd : **Transfer learning** is to adapt an existing model to recognize new types of objects instead of starting from scratch

# Use a trained model directly ro image recognition VGG16

In [1]:
import numpy as np
from keras.preprocessing import image
from keras.applications import vgg16

#creat an instance of the model
model = vgg16.VGG16()

553476096/553467096 [==============================] - 4s 0us/step


In [5]:
#load the image to the memory
img = image.load_img("bay.jpg", target_size = (224, 224))

the size of the image need to match the number of the input nodes in the neural network

For VGG16, images fed into the network need to be (224,224)

***caution: look at thses 2 ways to image-procesing***

In [11]:
#convert the image to a numpy array
x = image.img_to_array(img)

#add a forth dimaention, since keras expects a list of images WAY 1:
x1 = np.expand_dims(x, axis = 0)

x1.shape

(1, 224, 224, 3)

***Got the right output***

In [9]:
#convert the image to a numpy array
x = image.img_to_array(img)

#add a forth dimaention, since keras expects a list of images WAY 2:
x2 = np.array(list(x))

x2.shape

(224, 224, 3)

***Got the wrong output***

In [13]:
#normailze the images's pixals value to a range of 0-1
x = vgg16.preprocess_input(x1)

In [14]:
#run the image through the deep neural network to make prediction
prediction = model.predict(x)

This function returns a 1,000 element array, each element tells us how likely our picture contains each of the one hundered objects

In [17]:
#vgg16 de-code prediction function to tell us the names of the objects
#by default, it gives ut the 5 most likely objects 

predected_classes = vgg16.decode_predictions(prediction, top = 9)
print("Top predictions for this image: ")

for imagenet_id, name, likelihood in predected_classes[0]:
  print('Predictions : {} - {:2f}'.format(name , likelihood))

Top predictions for this image: 
Predictions : seashore - 0.423801
Predictions : promontory - 0.242124
Predictions : lakeside - 0.240130
Predictions : breakwater - 0.049119
Predictions : sandbar - 0.032311
Predictions : dock - 0.002331
Predictions : cliff - 0.002244
Predictions : beacon - 0.002014
Predictions : valley - 0.001612


# Transfer learning (feature extraction)

In [34]:
import joblib #used for domping the features as .dat file
from pathlib import Path
from keras.preprocessing import image
from keras.applications import vgg16
import numpy as np

#path to folders with training data
dog = Path()/"dogs"
not_dog = Path()/'not_dogs'

images = []
labels = []

for img in not_dog.glob("*.png"):
  img = image.load_img(img) #load the img in the memory
  img_array = image.img_to_array(img) #convert in into the array
  images.append(img_array) #add to the list
  labels.append(0) # add zero to tell it's not a dog

for img in dog.glob("*.png"):
  img = image.load_img(img)
  img_array = image.img_to_array(img)
  images.append(img_array)
  labels.append(1)

In [35]:
#keras expects the input to be a numpy array, so 
x_train = np.array(images)

#convert the labels to an array
y_train_ = np.array(labels)

#normalize the images array
x_train = vgg16.preprocess_input(x_train)

We wont convert the y_train to categorical, why?
'cuz this is a binary classification, we only need one element (0 or 1) for each input image

weights = 'imagenet', we tell keras that we wanna load the virsion with the neural network pretrained on the 'imagenet' dataset

Since we only use it for featrue extraction, we wanna chop-off the last layer of the neural network 
include_top = 'False'

In [36]:
#load a pre-trained neural network to use as feature extractor
pretrained_nn = vgg16.VGG16(weights = 'imagenet', include_top = False, input_shape= (64,64,3))
feature_extracted = pretrained_nn.predict(x_train)

the feature_extracted array now contain all the features in our training images

# Transfer learning (New NN traied with the extracted features)

In [37]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

x_train = feature_extracted[:40]
y_train = y_train_[:40]

x_test = feature_extracted[40:]
y_test = y_train_[40:]


x_train.shape[1:]
#x_train.shape returns an array, we use the last 3 elements by indexing [1:]

(2, 2, 512)

In [38]:
#create the model
model = Sequential()

model.add(Flatten(input_shape= x_train.shape[1:]))
model.add(Dense(256, activation= 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation = 'sigmoid'))

#compile the model
model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics= ['accuracy'])

#train the model
history = model.fit(x_train, y_train, epochs = 10)

Epoch 1/10
2/2 [==============================] - 0s 10ms/step - loss: 13.3864 - accuracy: 0.4500
Epoch 2/10
2/2 [==============================] - 0s 10ms/step - loss: 3.7283 - accuracy: 0.8750
Epoch 3/10
2/2 [==============================] - 0s 9ms/step - loss: 3.5951 - accuracy: 0.9000
Epoch 4/10
2/2 [==============================] - 0s 8ms/step - loss: 3.7610e-13 - accuracy: 1.0000
Epoch 5/10
2/2 [==============================] - 0s 8ms/step - loss: 9.7185e-11 - accuracy: 1.0000
Epoch 6/10
2/2 [==============================] - 0s 20ms/step - loss: 3.0179e-08 - accuracy: 1.0000
Epoch 7/10
2/2 [==============================] - 0s 10ms/step - loss: 1.2602e-12 - accuracy: 1.0000
Epoch 8/10
2/2 [==============================] - 0s 11ms/step - loss: 5.0400e-16 - accuracy: 1.0000
Epoch 9/10
2/2 [==============================] - 0s 11ms/step - loss: 4.2011e-09 - accuracy: 1.0000
Epoch 10/10
2/2 [==============================] - 0s 9ms/step - loss: 1.3834e-06 - accuracy: 1.0000


In [40]:
scores = model.evaluate(x_test,y_test)
print('accuracy : ', scores[1])

1/1 [==============================] - 0s 18ms/step - loss: 2.3779e-09 - accuracy: 1.0000
accuracy :  1.0
